<table style="width: 100%;"> <tr> <td style="width: 20%; vertical-align: top;"> <img src="https://upload.wikimedia.org/wikipedia/commons/archive/f/fb/20161010213812%21Escudo-UdeA.svg" alt="UdeA" height="150px"> </td> <td style="width: 80%; padding-left: 20px;"> <strong style="font-size: 30px;">EXPLORACIÓN DEL CONJUNTO DE DATOS PARA EL DESARROLLO DE UN SISTEMA DE RECOMENDACIÓN DE PELÍCULAS - PROYECTO MARKETING</strong><br><br> <span style="font-size: 30px;"> Luisa Fernanda Alzate Cuartas <br> Juan Camilo Henao Caro<br>Isabella Mendez Hoyos<br> Fernando Antonio Piñeres Ramos </span> </td> </tr> </table>

#### LIBRERÍAS REQUERIDAS

In [46]:
#Conexión de Google Colab con Drive.
from google.colab import drive
drive.mount('/content/drive')

#Acceder a los archivos de la carpeta marketing para el uso de funciones.
import sys
sys.path.append('/content/drive/MyDrive/marketing/Marketing')

Mounted at /content/drive


In [47]:
import pandas as pd # Manipulación y el analísis de datos.
import sqlite3 as sql # Interactuar con base de datos SQL usando Python.
from sklearn.preprocessing import MultiLabelBinarizer

#### CONEXIÓN CON LA BASE DE DATOS

In [48]:
#Establece una conexión a la base de datos SQLite ubicada en "content/db_movies"
conexion = sql.connect("/content/drive/MyDrive/marketing/Marketing/data/db_movies")

# Crear un cursor a partir de la conexión, que permita ejecutar comando SQL sobre la base de datos.
cur = conexion.cursor()

In [49]:
# Ejecuta una consulta SQL que obtiene los nombres de todas las tablas existentes en la base de datos.
cur.execute("SELECT name FROM sqlite_master where type='table' ")

# Recupera todos los resultados de la consulta anterior en forma de lista de tuplas.
cur.fetchall()

[('ratings',), ('movies',), ('ratings_with_date',)]

#### EXPLORACIÓN DE LAS TABLAS

##### TABLA MOVIES

In [50]:
# Ejecuta la consulta SQL "SELECT * FROM movies" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_movies = pd. read_sql_query("SELECT * FROM movies", conexion)
table_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy





```
# Transformaciones requeridas en la tabla movies
  - Separar el nombre de la película y el año de estreno.
  - Separación de los generos.
```





In [51]:
table_movies['genres'] = table_movies['genres'].str.split('|')

# Creamos el binarizador que convertirá las listas de géneros en columnas con 0 y 1
mlb = MultiLabelBinarizer()

# Aplicamos el binarizador a la columna 'genero' que contiene listas
generos_bin = mlb.fit_transform(table_movies['genres'])

# Creamos un nuevo DataFrame con esa matriz y ponemos como nombres de columna los géneros encontrados
df_generos = pd.DataFrame(generos_bin, columns=mlb.classes_)

# Unimos el DataFrame original (df_movies) con el nuevo DataFrame de géneros binarizados
df_movies = pd.concat([table_movies, df_generos], axis=1)

# DataFrame final con las nuevas columnas ya agregadas
display(df_movies.head())

,movieId,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]",0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),"[Comedy, Romance]",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),[Comedy],0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [52]:
# Eliminamos la columna 'genres' porque ya no es necesaria
df_movies = df_movies.drop(columns=['genres'])

display(df_movies)


,movieId,title,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,Documentary,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),0,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9738,193583,No Game No Life: Zero (2017),0,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9739,193585,Flint (2017),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9740,193587,Bungo Stray Dogs: Dead Apple (2018),0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [53]:
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   movieId             9742 non-null   int64 
 1   title               9742 non-null   object
 2   (no genres listed)  9742 non-null   int64 
 3   Action              9742 non-null   int64 
 4   Adventure           9742 non-null   int64 
 5   Animation           9742 non-null   int64 
 6   Children            9742 non-null   int64 
 7   Comedy              9742 non-null   int64 
 8   Crime               9742 non-null   int64 
 9   Documentary         9742 non-null   int64 
 10  Drama               9742 non-null   int64 
 11  Fantasy             9742 non-null   int64 
 12  Film-Noir           9742 non-null   int64 
 13  Horror              9742 non-null   int64 
 14  IMAX                9742 non-null   int64 
 15  Musical             9742 non-null   int64 
 16  Mystery             9742

In [54]:
df_movies.isnull().sum()

,0
movieId,0
title,0
(no genres listed),0
Action,0
Adventure,0
Animation,0
Children,0
Comedy,0
Crime,0
Documentary,0


In [55]:
total_observaciones = df_movies.size
total_nulos = df_movies.isnull().sum().sum()
porcentaje_nulos = (total_nulos / total_observaciones) * 100
print(f" el porcentaje de datos nulos de la base de datos de movies es: {porcentaje_nulos:.2f}%")

 el porcentaje de datos nulos de la base de datos de movies es: 0.00%


In [57]:


duplicados = df_movies.duplicated().sum()
print(f"Cantidad de duplicados: {duplicados}")


Cantidad de duplicados: 0


Transformaciones necesarias para la tabla movies: separar de la columna title el nombre de la película de la fecha de estreno y separar los generos.

In [ ]:
pd.read_sql_query("""
select * from movies where title like '%Goodnight Mommy%'

""", conexion)


##### TABLA RATINGS

In [64]:
# Ejecuta la consulta SQL "SELECT * FROM ratings" sobre la base de datos y carga los resultados en un DataFrame de pandas.
table_ratings = pd.read_sql_query("SELECT * FROM ratings", conexion)
table_ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


Transformaciones necesarias para la tabla ratings: Transformar la columna timestamp en formato fecha.

In [66]:
table_ratings['timestamp'] = pd.to_datetime(table_ratings['timestamp'], unit='s')


table_ratings['day'] = table_ratings['timestamp'].dt.day
table_ratings['month'] = table_ratings['timestamp'].dt.month
table_ratings['hour'] = table_ratings['timestamp'].dt.hour
table_ratings['year'] = table_ratings['timestamp'].dt.year



if 'date' in table_ratings.columns:
  table_ratings = table_ratings.drop('date', axis=1)

# Mostrar el DataFrame actualizado
display(table_ratings.head())


,userId,movieId,rating,timestamp,day,month,hour,year
0,1,1,4.0,2000-07-30 18:45:03,30,7,18,2000
1,1,3,4.0,2000-07-30 18:20:47,30,7,18,2000
2,1,6,4.0,2000-07-30 18:37:04,30,7,18,2000
3,1,47,5.0,2000-07-30 19:03:35,30,7,19,2000
4,1,50,5.0,2000-07-30 18:48:51,30,7,18,2000


In [67]:
# Eliminamos la columna 'timestamp' porque ya no es necesaria
table_ratings.drop(columns=['timestamp'], inplace=True)

display(table_ratings)

,userId,movieId,rating,day,month,hour,year
0,1,1,4.0,30,7,18,2000
1,1,3,4.0,30,7,18,2000
2,1,6,4.0,30,7,18,2000
3,1,47,5.0,30,7,19,2000
4,1,50,5.0,30,7,18,2000
...,...,...,...,...,...,...,...
100831,610,166534,4.0,3,5,21,2017
100832,610,168248,5.0,3,5,22,2017
100833,610,168250,5.0,8,5,19,2017
100834,610,168252,5.0,3,5,21,2017


#### PREGUNTAS PARA EXPLORACIÓN GENERAL

¿Cuántas películas hay registradas en total?

¿Cuántos géneros diferentes existen?

¿Cuál es el género más común entre las películas?

¿Cuántas calificaciones (ratings) hay en total?

¿Cuántos usuarios distintos han dado calificaciones?

¿Qué película tiene más calificaciones?

¿Qué usuario ha dado más calificaciones?

¿Cuál es la distribución de las calificaciones? (¿hay más 5.0, 4.0, etc.?)

¿Existen películas sin ninguna calificación?

¿Cuál es el promedio de calificación por película?

¿Cuál es el promedio de calificación por usuario?

¿Cuál es el rango de años de las películas? (¿de qué año a qué año?)

#### PREGUNTAS PARA DETENCCIÓN DE PROBLEMAS O CALIDAD DE DATOS

¿Hay películas con el mismo nombre pero diferente movieId?

¿Hay calificaciones fuera del rango esperado (por ejemplo, menores a 0 o mayores a 5)?

¿Existen valores nulos o vacíos en alguna columna?

¿Hay géneros desconocidos o no estandarizados?

¿Hay películas sin año de estreno?

#### PREGUNTAS PARA VISUALIZACIÓN DE DISTRIBUCIONES

Cómo es la distribución de las calificaciones?

¿Cómo es la distribución del número de calificaciones por película?

¿Cómo es la distribución del número de calificaciones por usuario?

¿Cómo varía la calificación promedio a lo largo de los años?

#### PREGUNTA ANÁLISIS AVANZADO



(Cruzar tablas y buscar patrones)

¿Cuál es el género que tiene mejores calificaciones promedio?

¿Las películas más recientes tienen mejores calificaciones que las antiguas?

¿Qué usuarios tienden a calificar más bajo o más alto?

¿Hay algún patrón entre género y año de estreno?

¿Cuál es la película mejor calificada?

¿Cuál es la película peor calificada?

¿Existe algún año donde se estrenaron muchas películas de un género específico?

¿Cómo cambia la popularidad de géneros a lo largo del tiempo?